In [2]:
!pip install pgmpy

     |████████████████████████████████| 1.9 MB 5.0 MB/s 


In [3]:
# Import libraries
import pgmpy.models
import pgmpy.inference
import numpy as np
# The main entry point for this module
def main():
    # Create a dynamic bayesian network
    model = pgmpy.models.DynamicBayesianNetwork()
    # Add nodes
    model.add_nodes_from(['Weather', 'Umbrella'])
    # Print nodes
    print('--- Nodes ---')
    print(model.nodes())
    # Add edges
    model.add_edges_from([(('Umbrella',0), ('Weather',0)),
                          (('Weather',0), ('Weather',1)),
                          (('Umbrella',0), ('Umbrella',1))])
    # Print edges
    print('--- Edges ---')
    print(model.edges())
    print()
    # Print parents
    print('--- Parents ---')
    print('Umbrella 0: {0}'.format(model.get_parents(('Umbrella', 0))))
    print('Weather 0: {0}'.format(model.get_parents(('Weather', 0))))
    print('Weather 1: {0}'.format(model.get_parents(('Weather', 1))))
    print('Umbrella 1: {0}'.format(model.get_parents(('Umbrella', 1))))
    print()
    # Add probabilities
    weather_cpd = pgmpy.factors.discrete.TabularCPD(('Weather', 0), 2, [[0.1, 0.8], 
                                                                        [0.9, 0.2]], 
                                                       evidence=[('Umbrella', 0)], 
                                                       evidence_card=[2])
    umbrella_cpd = pgmpy.factors.discrete.TabularCPD(('Umbrella', 1), 2, [[0.5, 0.5], 
                                                                          [0.5, 0.5]], 
                                                     evidence=[('Umbrella', 0)], 
                                                     evidence_card=[2])
    transition_cpd = pgmpy.factors.discrete.TabularCPD(('Weather', 1), 2, [[0.25, 0.9, 0.1, 0.25], 
                                                                           [0.75, 0.1, 0.9, 0.75]], 
                                                   evidence=[('Weather', 0), ('Umbrella', 1)], 
                                                   evidence_card=[2, 2])
    # Add conditional probability distributions (cpd:s)
    model.add_cpds(weather_cpd, umbrella_cpd, transition_cpd)
    # This method will automatically re-adjust the cpds and the edges added to the bayesian network.
    model.initialize_initial_state()
    # Check if the model is valid, throw an exception otherwise
    model.check_model()
    # Print probability distributions
    print('Probability distribution, P(Weather(0) | Umbrella(0)')
    print(weather_cpd)
    print()
    print('Probability distribution, P(Umbrella(1) | Umbrella(0)')
    print(umbrella_cpd)
    print()
    print('Probability distribution, P(Weather(1) | Umbrella(1), Weather(0)')
    print(transition_cpd)
    print()
    # Make inference
    map = {0: 'Sunny', 1: 'Rainy' }
    dbn_inf = pgmpy.inference.DBNInference(model)
    result = dbn_inf.forward_inference([('Weather', 1)], {('Umbrella', 1):0, ('Weather', 0):0})
    arr = result[('Weather', 1)].values
    print()
    print('Prediction (Umbrella(1) : Yes, Weather(0): Sunny): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
    print()
    result = dbn_inf.forward_inference([('Weather', 1)], {('Umbrella', 1):0, ('Weather', 0):1})
    arr = result[('Weather', 1)].values
    print()
    print('Prediction (Umbrella(1) : Yes, Weather(0): Rainy): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
    print()
    result = dbn_inf.forward_inference([('Weather', 1)], {('Umbrella', 1):1, ('Weather', 0):0})
    arr = result[('Weather', 1)].values
    print()
    print('Prediction (Umbrella(1) : No, Weather(0): Sunny): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
    print()
    result = dbn_inf.forward_inference([('Weather', 1)], {('Umbrella', 1):1, ('Weather', 0):1})
    arr = result[('Weather', 1)].values
    print()
    print('Prediction (Umbrella(1) : No, Weather(0): Rainy): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
    print()
# Tell python to run main method
if __name__ == "__main__": main()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


--- Nodes ---
[<DynamicNode(Weather, 0) at 0x7ff9bb8c9f10>, <DynamicNode(Umbrella, 0) at 0x7ff9ad143a10>]
--- Edges ---
[(<DynamicNode(Weather, 0) at 0x7ff9bb8c9f10>, <DynamicNode(Weather, 1) at 0x7ff99d1b4e50>), (<DynamicNode(Umbrella, 0) at 0x7ff9ad143a10>, <DynamicNode(Weather, 0) at 0x7ff99d1b4d10>), (<DynamicNode(Umbrella, 0) at 0x7ff9ad143a10>, <DynamicNode(Umbrella, 1) at 0x7ff99d1b4dd0>), (<DynamicNode(Umbrella, 1) at 0x7ff99d1b4d90>, <DynamicNode(Weather, 1) at 0x7ff99d1b4d50>)]

--- Parents ---
Umbrella 0: []
Weather 0: [<DynamicNode(Umbrella, 0) at 0x7ff99d1b4cd0>]
Weather 1: [<DynamicNode(Umbrella, 1) at 0x7ff99d1b4d90>, <DynamicNode(Weather, 0) at 0x7ff99d1b4e10>]
Umbrella 1: [<DynamicNode(Umbrella, 0) at 0x7ff99d1b4e90>]

Probability distribution, P(Weather(0) | Umbrella(0)
+-------------------+--------------------+--------------------+
| ('Umbrella', 0)   | ('Umbrella', 0)(0) | ('Umbrella', 0)(1) |
+-------------------+--------------------+--------------------+
| ('Weath

  0%|          | 0/1 [00:00<?, ?it/s]


Prediction (Umbrella(1) : Yes, Weather(0): Sunny): Rainy (75.0 %)



  0%|          | 0/1 [00:00<?, ?it/s]


Prediction (Umbrella(1) : Yes, Weather(0): Rainy): Rainy (90.0 %)



  0%|          | 0/1 [00:00<?, ?it/s]


Prediction (Umbrella(1) : No, Weather(0): Sunny): Sunny (90.0 %)



  0%|          | 0/1 [00:00<?, ?it/s]


Prediction (Umbrella(1) : No, Weather(0): Rainy): Rainy (75.0 %)



In [ ]:
#--------Q2--------#

class SocialNetwork:
    
    def __init__(self):
        self.graph = {}
        
    def addFriend(self, person, friend):
        
        if person not in self.graph:
            self.graph[person] = []
            
        self.graph[person].append(friend)

        if friend not in self.graph:
            self.graph[friend] = []
            
        self.graph[friend].append(person)
        
    def getFriendCounts(self):
        
        count = {}
        for key in self.graph:
            count[key] = len(self.graph[key])
            
        return count
    
    def mostSocialStudents(self):
        
        count = self.getFriendCounts()
        
        students = []
        
        for person in count:
            
            if count[person] > 10:
                students.append(person)
                
        return students
        

agent = SocialNetwork()

agent.addFriend('Huzaifa', 'Farooq')
agent.addFriend('Huzaifa', 'Hamza')
agent.addFriend('Hamza', 'Farooq')
agent.addFriend('Ahmad', 'Basit')
agent.addFriend('Ahmad', 'Zohaib')
agent.addFriend('Ahmad', 'Daniyal')
agent.addFriend('Ahmad', 'Muneeb')
agent.addFriend('Ahmad', 'Farooq')
agent.addFriend('Ahmad', 'Hamza')
agent.addFriend('Ahmad', 'Huzaifa')
agent.addFriend('Ahmad', 'Vinesh')
agent.addFriend('Ahmad', 'Jay')
agent.addFriend('Ahmad', 'Samad')
agent.addFriend('Ahmad', 'Saif')
agent.addFriend('Jay', 'Vinesh')
agent.addFriend('Samad', 'Jay')
agent.addFriend('Saif', 'Basit')
agent.addFriend('Saif', 'Zohaib')
agent.addFriend('Zohaib', 'Daniyal')
agent.addFriend('Basit', 'Muneeb')
agent.addFriend('Daniyal', 'Muneeb')

print('Most Social Students in the University is:', agent.mostSocialStudents())
print('Social Network:')

network = agent.graph

for person in network:
    print(person, '->', network[person])



Most Social Students in the University is: ['Ahmad']
Social Network:
Huzaifa -> ['Farooq', 'Hamza', 'Ahmad']
Farooq -> ['Huzaifa', 'Hamza', 'Ahmad']
Hamza -> ['Huzaifa', 'Farooq', 'Ahmad']
Ahmad -> ['Basit', 'Zohaib', 'Daniyal', 'Muneeb', 'Farooq', 'Hamza', 'Huzaifa', 'Vinesh', 'Jay', 'Samad', 'Saif']
Basit -> ['Ahmad', 'Saif', 'Muneeb']
Zohaib -> ['Ahmad', 'Saif', 'Daniyal']
Daniyal -> ['Ahmad', 'Zohaib', 'Muneeb']
Muneeb -> ['Ahmad', 'Basit', 'Daniyal']
Vinesh -> ['Ahmad', 'Jay']
Jay -> ['Ahmad', 'Vinesh', 'Samad']
Samad -> ['Ahmad', 'Jay']
Saif -> ['Ahmad', 'Basit', 'Zohaib']
